# Demo of SED and photometry calculations with DSPS
author: Luca Tortorelli, Andrew Hearin

last run successfully: Nov 7, 2022

This notebook demonstrates some basic usage of the DSPS library. In particular, for a galaxy with some arbitrary star formation history, we'll see how to calculate its restframe SED, and its absolute and apparent magnitude in some band.

Andrew:
Warning: At present, DSPS is mostly just a collection of JAX kernels and does not have much in the way of a user-interface or documentation. The API shown here is very likely to evolve in the coming year. I expect that a stable interface with end-user docs will not arrive until spring 2023 at the earliest.

SPS calculations are based on a set of template SEDs of simple stellar populations (SSPs). Supplying such templates is outside the planned scope of the DSPS package, and so they will need to be retrieved from some other library. For example, the FSPS library supplies such templates in a convenient form.

In [ ]:
import rail
from rail.core.stage import RailStage
from rail.creation.engines.dsps_sed_modeler import DSPSSingleSedModeler, DSPSPopulationSedModeler
from rail.creation.engines.dsps_photometry_creator import DSPSPhotometryCreator
from dsps.load_fsps_data import load_fsps_testing_data
import numpy as np
import os
from dsps.utils import _jax_get_dt_array

In [ ]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

The data that are required to generate a single SED model are stored in /src/rail/examples_data/testdata and can be generated via this notebook. They are loaded from .npy files. The required data are:
- the fluxes of the single stellar populations (SSP), ssp_fluxes
- the corresponding SSP wavelengths, ssp_wavelengths
- the age grid of the SSP in units of log10(Age[Gyr]), age_grid
- the metallicity grid of the SSP in units of log10(Z*/Z*_solar), metallicity_grid
- the galaxy star-formation history in units of log10(M*/M*_solar) per time step in units of Gyr, star_formation_history
- the cosmic time age grid up to the Universe age in Gyr, cosmic_time_grid
- keyword to choose between total formed stellar mass or surviving stellar mass, stellar_mass_type
- the galaxy stellar mass build-up table to-be-used when stellar_mass_type="surviving", stellar_mass_table
- galaxy age at the time of observation in Gyr, galaxy_age
- galaxy metallicity at the time of observation in log10(Z*/Z*_solar), galaxy_metallicity
- log normal scatter of the galaxy metallicity at the time of observation, galaxy_metallicity_scatter

To better clarify the meaning of the parameters above, we provide an example of how to generate the required npy files

In [ ]:
# the SSP fluxes and wavelengths were provided by Andrew and generated using FSPS
DRN = '/src/rail/examples_data/testdata/'
ssp_flux = np.load(os.path.join(DRN, "dsps_ssp_spec_flux.npy"))
ssp_wave = np.load(os.path.join(DRN, "dsps_ssp_spec_wave.npy"))

In [ ]:
# DSPS contains a function to load the grid of ages and metallicities from SSPs created with FSPS, as well as the LSST filters
# filter_data, __, lgZsun_bin_mids, log_age_gyr = load_fsps_testing_data(DRN)

In [ ]:
# the grid of ages and metallicities should reflect those used to generate the SSP fluxes
log_age_gyr = np.arange(-3.5, 1.20, 0.05, dtype='float64')
lgZsun_bin_mids = np.array([-1.97772361, -1.80163235, -1.67669361, -1.5797836, -1.50060235,
                            -1.37566361, -1.2787536, -1.19957235, -1.07463362, -0.97772361,
                            -0.88081359, -0.78739191, -0.68768899, -0.58855752, -0.49342377,
                            -0.39226288, -0.29648237, -0.19957235, -0.10266234, 0., 0.10145764, 0.19836765])

In [ ]:
# optionally save the age and metallicity grids in npy files
np.save(os.path.join(DRN, 'metallicity_grid.npy'), lgZsun_bin_mids)
np.save(os.path.join(DRN, 'age_grid.npy'), log_age_gyr)

In [ ]:
# create the SFH and stellar mass build-up of a dummy galaxy
n_t = 100 # number of cosmic age time steps
T0 = 13.8 # Universe age today
t_table = np.linspace(0.1, T0, n_t)
dt_table = _jax_get_dt_array(t_table)
sfh_table = np.random.uniform(0, 10, t_table.size) # star-formation history with units as above
logsm_table = np.log10(np.cumsum(sfh_table*dt_table)) + 9.0 # total formed stellar mass

In [ ]:
# optionally save data in npy files
np.save(os.path.join(DRN, 'SFH.npy'), sfh_table)
np.save(os.path.join(DRN, 'cosmic_time_table.npy'), t_table)
np.save(os.path.join(DRN, 'stellar_mass_table.npy'), logsm_table)

### Class initialization for the creation of single SED model with DSPS

In [ ]:
single_sed_model = DSPSSingleSedModeler.make_stage(name='DSPS_single_SED_model', age_grid=os.path.join(DRN, 'age_grid.npy'),
                                                   metallicity_grid=os.path.join(DRN, 'metallicity_grid.npy'),
                                                   ssp_fluxes=os.path.join(DRN, "dsps_ssp_spec_flux.npy"),
                                                   star_formation_history=os.path.join(DRN, 'SFH.npy'),
                                                   cosmic_time_grid=os.path.join(DRN, 'cosmic_time_table.npy'),
                                                   stellar_mass_type='formed',
                                                   stellar_mass_table=os.path.join(DRN, 'stellar_mass_table.npy'),
                                                   galaxy_age=7, galaxy_metallicity=0.0,
                                                   galaxy_metallicity_scatter=0.2)

Run() and fit_model() to generate the actual rest-frame SED in units of L_solar/Hz. Since the output is a ModelHandle, the rest-frame SED is stored into a .pkl file

In [ ]:
data_handle = single_sed_model.fit_model()

In [ ]:
print(data_handle.data)

### Class initialization for the creation of a population of SEDs with DSPS

In case we would like to generate a population of model rest-frame SEDs, we need to invoke the DSPSPopulationSedModeler class. In this case, the npy files for the star-formation history, cosmic time grid and stellar mass build-up array have shape (n_gal_population, n_features), where n_features is the shape of the arrays in the single SED case. Additionally, now also the galaxy age, metallicity and log normal scatter need to be passed as npy files storing arrays of shape (n_gal_populations). Examples npy files for this case are also stored in /src/rail/examples_data/testdata/ and generated in this notebook.

In [ ]:
# let's define a number of galaxies part of the galaxy population and use the previous star-formation histories and stellar masses build-up
# to create arrays for the population

n_gal_population = 10

sfhs_table = np.empty((n_gal_population, len(sfh_table)))
for i in range(n_gal_population): sfhs_table[i, :] = sfh_table

ts_table = np.empty((n_gal_population, len(t_table)))
for i in range(n_gal_population): ts_table[i, :] = t_table

logsms_table = np.empty((n_gal_population, len(logsm_table)))
for i in range(n_gal_population): logsms_table[i, :] = logsm_table

galaxy_ages = np.random.uniform(low=1, high=13, size=n_gal_population)
galaxy_metallicities = np.random.choice(lgZsun_bin_mids, size=n_gal_population, replace=False)
galaxy_metallicity_scatters = np.random.normal(loc=0.2, scale=0.1, size=n_gal_population)

In [ ]:
# optionally save data in npy files
np.save(os.path.join(DRN, 'SFHs.npy'), sfhs_table)
np.save(os.path.join(DRN, 'cosmic_times_table.npy'), ts_table)
np.save(os.path.join(DRN, 'stellar_masses_table.npy'), logsms_table)
np.save(os.path.join(DRN, 'galaxy_population_ages.npy'), galaxy_ages)
np.save(os.path.join(DRN, 'galaxy_population_metallicities.npy'), galaxy_metallicities)
np.save(os.path.join(DRN, 'galaxy_population_metallicity_scatters.npy'), galaxy_metallicity_scatters)

In [ ]:
population_sed_model = DSPSPopulationSedModeler.make_stage(name='model_DSPS_pop_sed_model',
                                                           age_grid=os.path.join(DRN, 'age_grid.npy'),
                                                           metallicity_grid=os.path.join(DRN, 'metallicity_grid.npy'),
                                                           ssp_fluxes=os.path.join(DRN, "dsps_ssp_spec_flux.npy"),
                                                           star_formation_history=os.path.join(DRN, 'SFHs.npy'),
                                                           cosmic_time_grid=os.path.join(DRN, 'cosmic_times_table.npy'),
                                                           stellar_mass_type='formed',
                                                           stellar_mass_table=os.path.join(DRN, 'stellar_masses_table.npy'),
                                                           galaxy_age=os.path.join(DRN, 'galaxy_population_ages.npy'),
                                                           galaxy_metallicity=os.path.join(DRN, 'galaxy_population_metallicities.npy'),
                                                           galaxy_metallicity_scatter=os.path.join(DRN, 'galaxy_population_metallicity_scatters.npy'))

In [ ]:
model_handle = population_sed_model.fit_model()

In [ ]:
print(model_handle.data)

### Observed photometry generation with DSPS

Generating the observed photometry with DSPS is simple and requires only few input from the user in npy file format, except for the ModelHandle from the SedModeler classes that are stored in .pkl files and generated with the classes above. The required data are:
- filter wavebands, filter_data
- rest-frame sed models, rest_frame_sed_models
- galaxy redshifts, galaxy_redshifts
- Omega matter today, Om0
- Omega lambda today,
- dark energy equation of state today's value, w0
- dark energy equation of state linear slope, wa
- Hubble parameter h = H_0/100, h
- keywork to set the previous cosmological parameters to Planck values, use_planck_cosmology
- number of galaxies in the population, n_galaxies

In [ ]:
# keeping the number of galaxies in the population fixed at the value above, let's generate a uniform distribution of
# galaxy redshifts between 0.1 and 1
galaxy_redshifts = np.random.uniform(0.1, 1.1, n_gal_population)

# optionally save data in npy format
np.save(os.path.join(DRN, 'galaxy_redshifts.npy'), galaxy_redshifts)

In [ ]:
n_gal_population = 10
phot_creator = DSPSPhotometryCreator.make_stage(filter_data=os.path.join(DRN, 'lsst_filters.npy'),
                                                rest_frame_sed_models='model_DSPS_pop_sed_model.pkl',
                                                rest_frame_wavelengths=os.path.join(DRN, "dsps_ssp_spec_wave.npy"),
                                                galaxy_redshifts=os.path.join(DRN, 'galaxy_redshifts.npy'),
                                                Om0=0.3, Ode0=0.7, w0=-1, wa=0, h=0.7,
                                                use_planck_cosmology=True, n_galaxies=n_gal_population)

The sample() method computes the absolute magnitudes and the observed magnitudes in the AB system for all the provided filters and for all the galaxies and store the results in a fits table

In [ ]:
out_table = phot_creator.sample()

In [ ]:
print(out_table.data)